In [3]:
pip install numpy -q

In [4]:
pip install pandas -q

In [5]:
pip install scikit-learn -q

In [6]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
import seaborn as sns

In [7]:
df = sns.load_dataset('iris')
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

In [8]:
df["species"].replace({"setosa": 0, "versicolor": 1, "virginica": 2}, inplace=True)

# Renomeando as colunas
df.columns = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Class"]

# Separando os labels (classes)
labels = df["Class"].values
df.drop("Class", axis=1, inplace=True)

<ipython-input-8-1d150e0641c7>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["species"].replace({"setosa": 0, "versicolor": 1, "virginica": 2}, inplace=True)
<ipython-input-8-1d150e0641c7>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["species"].replace({"setosa": 0, "versicolor": 1, "vi

In [9]:
dados = df.to_numpy()
dados

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

### Inicialização da matriz de pertinência

A matriz de pertinência é inicializada aleatoriamente $u_{ijk}(i=1,...c$; $j=1,...,p$ e $k=1,...,n)$ do objeto $k$ pertencente ao grupo $C_i$ para a variável $j$ tal que:
- $u_{ijk} \in [0,1]$;
- $0 < \sum_{k=1}^nu_{ijk} < n$;
- $\sum_{i=1}^c\sum_{j=1}^pu_{ijk} = 1$ para todo $k \in \Omega$.

In [13]:
def inicializao_matriz_pertinencia(num_amostras, num_clusters, num_variaveis):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters, num_variaveis) # gera uma matriz inicial aleatória com valores entre 0 e 1
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return matriz_pertinencia

### Atualização dos centroides

Fixo os graus de pertinência, os centroides são atualizados com base nessa equação:

### $y_{ij} = \frac{\sum_{k=1}^n(u_{ijk})^mx_{kj}}{\sum_{k=1}^n(u_{ijk})^m}$

In [ ]:
def atualizacao_centroides(dados, matriz_pertinencia, m):
    num_clusters = matriz_pertinencia.shape[1]
    num_variaveis = dados.shape[1]

    matriz_pertinencia_m = matriz_pertinencia ** m
    centroides = np.zeros((num_clusters, num_variaveis))

    for j in range(num_variaveis):  # para cada variável
        for i in range(num_clusters):  # para cada cluster
            numerador = np.sum(matriz_pertinencia_m[:, i, j].reshape(-1, 1) * dados[:, j].reshape(-1, 1)) # por que não dados[i,j]?
            denominador = np.sum(matriz_pertinencia_m[:, i, j])
            centroides[i, j] = numerador / denominador

    return centroides